In [73]:
import pandas as pd
import copy
import numba
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 252 ms


In [2]:
game_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "2016_17_NBA_Scores")
team_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "Division_Info")

time: 158 ms


In [3]:
def create_game_dicts(game_data):
    games = []
    for index, row in game_data.iterrows():
        game = {}
        if row['Winner'] == 'Home':
            game[row['Home Team']] = 'Winner'
            game[row['Away Team']] = 'Loser'
        else:
            game[row['Home Team']] = 'Loser'
            game[row['Away Team']] = 'Winner'
        games.append(game)
    return games

time: 4.78 ms


In [4]:
def simulate_games(games, teams):
    for game in games:
        for team in game:
            teams[team]['Games_Played'] += 1
            if game[team] == 'Loser':
                teams[team]['Max_Wins'] -= 1

time: 2.13 ms


In [5]:
def initialize_team_standings(team_data):
    teams = {}
    for index, row in team_data.iterrows():
        teams[row['Team_Name']] = {'Division': row['Division_id'], 'Conference': row['Conference_id'], 'Games_Played': 0, 
                                  'Max_Wins': 82}
    return teams

time: 2.15 ms


In [6]:
def get_league_data(team_data):
    east_divisions = set(team_data[team_data['Conference_id'] == 'East']['Division_id'])
    west_divisions = set(team_data[team_data['Conference_id'] == 'West']['Division_id'])
    east_teams = set(team_data[team_data['Conference_id'] == 'East']['Team_Name'])
    west_teams = set(team_data[team_data['Conference_id'] == 'West']['Team_Name'])
    return {'East': {'Divisions': east_divisions, 'Teams': east_teams}, 
            'West': {'Divisions': west_divisions, 'Teams': west_teams}}

time: 3.36 ms


In [54]:
def initialize_league_standings(team_data):
    league_standings = {'East': {'Conference': {}, 'Atlantic': {}, 'Central': {}, 'Southeast': {}}, 
                    'West': {'Conference': {}, 'Northwest': {}, 'Southwest': {}, 'Pacific': {}}}
    for index, row in team_data.iterrows():
        league_standings[row['Conference_id']]['Conference'][row['Team_Name']] = 82
        league_standings[row['Conference_id']][row['Division_id']][row['Team_Name']] = 82
    return league_standings

time: 3.9 ms


In [128]:
def get_playoff_teams(league_data, league_standings):
    playoff_teams_all = {}
    for conference in ['East', 'West']:
        division_leaders = []
        for division in league_data[conference]['Divisions']:
            team = max(league_standings[conference][division], key=league_standings[conference][division].get)
            division_leaders.append(team)
        for team, max_wins in league_standings[conference]['Conference'].items():
            standings = sorted(league_standings[conference]['Conference'], 
                               key=league_standings[conference]['Conference'].get, reverse=True)
            playoff_teams = [x for x in standings if x not in division_leaders][0:5] + division_leaders
        playoff_teams_all[conference] = playoff_teams
    return playoff_teams_all

time: 7 ms


In [131]:
class DayOfGames(object):
    def __init__(self, data, team_standings, league_data, league_standings):
        self.data = data
        self.league_data = league_data
        self.date = self.data.iloc[0,0]
        self.games = self.create_game_dicts()
        self.team_standings_universal = team_standings
        self.league_standings_universal = league_standings
        self.add_game_results()
        self.team_standings = copy.deepcopy(self.team_standings_universal)
        self.league_standings = copy.deepcopy(self.league_standings_universal)
        self.playoff_teams = self.get_playoff_teams()
        
    def create_game_dicts(self):
        games = []
        for index, row in self.data.iterrows():
            game = {}
            if row['Winner'] == 'Home':
                game[row['Home Team']] = 'Winner'
                game[row['Away Team']] = 'Loser'
            else:
                game[row['Home Team']] = 'Loser'
                game[row['Away Team']] = 'Winner'
            games.append(game)
        return games    
        
    def add_game_results(self):
        for game in self.games:
            for team in game:
                self.team_standings_universal[team]['Games_Played'] += 1
                if game[team] == 'Loser':
                    self.team_standings_universal[team]['Max_Wins'] -= 1
        for team, info in self.team_standings_universal.items():
            self.league_standings_universal[info['Conference']]['Conference'][team] = info['Max_Wins']
            self.league_standings_universal[info['Conference']][info['Division']][team] = info['Max_Wins']
            
    def get_playoff_teams(self):
        playoff_teams_all = {}
        for conference in ['East', 'West']:
            division_leaders = []
            for division in self.league_data[conference]['Divisions']:
                team = max(self.league_standings[conference][division], 
                           key=self.league_standings[conference][division].get)
                division_leaders.append(team)
            for team, max_wins in self.league_standings[conference]['Conference'].items():
                standings = sorted(self.league_standings[conference]['Conference'], 
                                   key=self.league_standings[conference]['Conference'].get, reverse=True)
                playoff_teams = [x for x in standings if x not in division_leaders][0:5] + division_leaders
            playoff_teams_all[conference] = playoff_teams
        return playoff_teams_all

time: 39.7 ms


In [132]:
league_data = get_league_data(team_data)
team_standings = initialize_team_standings(team_data)
league_standings = initialize_league_standings(team_data)
days = []
first_game_date = game_data.iloc[0,0]
first_game_index = 0
for index, row in game_data.iloc[0:].iterrows():
    if row['Date'] != first_game_date:
        days.append(DayOfGames(game_data.iloc[first_game_index:index], team_standings, league_data, league_standings))
        first_game_index = index
        first_game_date = row['Date']
days.append(DayOfGames(game_data.iloc[first_game_index:index+1], team_standings, league_data, league_standings))

time: 370 ms
